In [1]:
import numpy as np
import loguru
import sys

In [2]:
cd ..

d:\project\FedImpute


d:\project\FedImpute\.venv\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [15]:
%load_ext autoreload
%autoreload 2
from fedimpute.simulator import Simulator

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
data = np.random.rand(10000, 10)
data_config = {
    'task_type': 'regression',
    'clf_type': None,
    'num_cols': 9,
}

simulator = Simulator(debug_mode=False)
simulation_results = simulator.simulate_scenario(data, data_config, num_clients = 10, verbose=1)

Data partitioning...
Missing data simulation...
Simulation done. Using summary function to check the simulation results.


In [5]:
%load_ext autoreload
%autoreload 2
from fedimpute.execution_environment import FedImputeEnv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
env = FedImputeEnv()
env.configuration(imputer = 'gain', fed_strategy='fedavg', fit_mode = 'fed')
# env.setup(
#     clients_train_data=simulation_results['clients_train_data'],
#     clients_train_data_ms=simulation_results['clients_train_data_ms'],
#     clients_test_data=simulation_results['clients_test_data'],
#     clients_seeds=simulation_results['clients_seeds'],
#     data_config=data_config,
# )
env.setup_from_simulator(simulator = simulator, verbose=1)

Setting up clients...
Setting up server...
Setting up workflow...
Environment setup complete.


In [18]:
print(env.imputer_name)
print(env.fed_strategy_name)

gain
fedavg


In [19]:
env.run_fed_imputation()

2024-06-13 21:34:54.115 | INFO     | fedimpute.execution_environment.workflows.workflow:eval_and_track:58 - 
Initial: rmse - 0.5771345352186719 ws - 0.2822121360258175


Global Epoch:   0%|          | 0/150 [00:00<?, ?it/s]

2024-06-13 21:35:24.478 | INFO     | fedimpute.execution_environment.workflows.workflow_jm:fed_imp_sequential:150 - All clients have converged. Stopping training at 40.
2024-06-13 21:35:24.479 | INFO     | fedimpute.execution_environment.workflows.workflow_jm:fed_imp_sequential:168 - start fine tuning ...
local True


Fine Tuning Epoch: 0it [00:00, ?it/s]

2024-06-13 21:35:26.177 | INFO     | fedimpute.execution_environment.workflows.workflow:eval_and_track:108 - Final: rmse - 0.2881824275210135 ws - 0.08930803039756144


In [70]:
%load_ext autoreload
%autoreload 2
from fedimpute.evaluation import Evaluator

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
evaluator = Evaluator()
eval_ret = evaluator.evaluate(env, ['imp_quality', 'pred_downstream_local', 'pred_downstream_fed'])
evaluator.show_results()

Evaluating imputation quality...
Evaluating downstream prediction...


Clients:   0%|          | 0/10 [00:00<?, ?it/s]

Evaluating federated downstream prediction...


Global Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

Evaluation completed.


In [72]:
evaluator.show_results()

Evaluation Results
Imputation Quality             | rmse: 0.288 (0.00) nrmse: 0.712 (0.03) sliced-ws: 0.089 (0.01) 
Downstream Prediction (Local)  |  mse: 0.087 (0.01)   mae: 0.248 (0.02)      msle: 0.041 (0.01) 
Downstream Prediction (Fed)    |  mse: 0.093 (0.00)   mae: 0.264 (0.00)      msle: 0.044 (0.00) 
